In [1]:
import json #for using json library function library

In [2]:
val = 5 # to save 


In [3]:
memory = dict(val =val) #make dictionary i.e. key value pair

In [13]:
#save val into memory.json file
with open('memory.json','w') as file: 
    json.dumps(memory, file)

NameError: name 'json' is not defined

In [8]:
# loading back the memory json and loaded in the variable named memo
with open('memory.json','r') as file: 
    memo = json.load(file)

In [9]:
memo 

{u'val': 5}

In [11]:
memo['val'] # access the value using the key

5

## POS tagging supervised
reference:  https://nlpforhackers.io/lstm-pos-tagger-keras/

In [2]:
import nltk

In [3]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()

In [3]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /Users/aman/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [4]:
print(tagged_sentences[0])

[(u'Pierre', u'NNP'), (u'Vinken', u'NNP'), (u',', u','), (u'61', u'CD'), (u'years', u'NNS'), (u'old', u'JJ'), (u',', u','), (u'will', u'MD'), (u'join', u'VB'), (u'the', u'DT'), (u'board', u'NN'), (u'as', u'IN'), (u'a', u'DT'), (u'nonexecutive', u'JJ'), (u'director', u'NN'), (u'Nov.', u'NNP'), (u'29', u'CD'), (u'.', u'.')]


In [5]:
#dataset statistics
len(tagged_sentences)

3914

In [6]:
import numpy as np

In [7]:
sentences, sentence_tags = [],[]
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))
    

In [8]:
sentences[13],sentence_tags[13]

(array([u'The', u'Lorillard', u'spokeswoman', u'said', u'0', u'asbestos',
        u'was', u'used', u'*-1', u'in', u'``', u'very', u'modest',
        u'amounts', u"''", u'in', u'*', u'making', u'paper', u'for',
        u'the', u'filters', u'in', u'the', u'early', u'1950s', u'and',
        u'replaced', u'*-1', u'with', u'a', u'different', u'type', u'of',
        u'filter', u'in', u'1956', u'.'], dtype='<U11'),
 array([u'DT', u'NNP', u'NN', u'VBD', u'-NONE-', u'NN', u'VBD', u'VBN',
        u'-NONE-', u'IN', u'``', u'RB', u'JJ', u'NNS', u"''", u'IN',
        u'-NONE-', u'VBG', u'NN', u'IN', u'DT', u'NNS', u'IN', u'DT',
        u'JJ', u'CD', u'CC', u'VBN', u'-NONE-', u'IN', u'DT', u'JJ', u'NN',
        u'IN', u'NN', u'IN', u'CD', u'.'], dtype='<U6'))

In [9]:
from sklearn.model_selection import train_test_split

(train_x, test_x, train_y, test_y) = train_test_split(sentences, sentence_tags, test_size=0.2)

In [10]:
test_y[0]

array([u'CC', u'DT', u'NNS', u'VBP', u'NNP', u'NNP', u'POS', u'NN', u'NN',
       u':', u'PRP', u'VBZ', u'NNS', u'IN', u'NNS', u'DT', u'NN', u'CC',
       u'VBZ', u'IN', u'IN', u'NNP', u'POS', u'NN', u'NNS', u'.'],
      dtype='<U3')

In [11]:
words, tags = set([]), set([])

for s in train_x:
    for w in s:
        words.add(w.lower())
        
for ts in train_y:
    for t in ts:
        tags.add(t)
        
        
word2idx = {w: i+2 for i ,w in enumerate(list(words))}
word2idx['<PAD>'] = 0
word2idx['<UNK>'] = 1

tag2idx = {t : i+1 for i, t in enumerate(list(tags))}
tag2idx['<PAD>'] = 0

In [12]:
ntrain_x, ntest_x, ntrain_y, ntest_y = [],[],[],[]

for s in train_x:
    s_int = []
    for w in s:
        try:
            s_int.append(word2idx[w.lower()])
        except:
            s_int.append(word2idx['<UNK>'])
    ntrain_x.append(s_int)
    
for s in test_x:
    s_int = []
    for w in s:
        try:
            s_int.append(word2idx[w.lower()])
        except:
            s_int.append(word2idx['<UNK>'])
    ntest_x.append(s_int)
    
for s in train_y:
    ntrain_y.append([tag2idx[t] for t in s])
    
for s in test_y:
    ntest_y.append([tag2idx[t] for t in s])

In [13]:
len(ntrain_x[0]),len(ntrain_y[0])

(3, 3)

In [14]:
MAX_LENGTH = len(max(ntrain_x, key =len))
print(MAX_LENGTH)

271


In [15]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [16]:
ntrain_x = pad_sequences(ntrain_x, maxlen=MAX_LENGTH, padding = 'post')
ntest_x = pad_sequences(ntest_x, maxlen=MAX_LENGTH, padding='post')
ntrain_y = pad_sequences(ntrain_y, maxlen=MAX_LENGTH, padding = 'post')
ntest_y = pad_sequences(ntest_y, maxlen=MAX_LENGTH, padding='post')

In [17]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2idx), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2idx))))
model.add(Activation('softmax'))

model.compile(loss ='categorical_crossentropy',
             optimizer=Adam(0.001),
             metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 271, 128)          1307264   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 271, 512)          788480    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 271, 47)           24111     
_________________________________________________________________
activation_1 (Activation)    (None, 271, 47)           0         
Total params: 2,119,855
Trainable params: 2,119,855
Non-trainable params: 0
_________________________________________________________________


In [41]:
# def to_categorical(sequences, categories):
#     cat_sequences = []
#     for s in sequences:
#         cats =[]
#         for item in s:
#             cats.append(np.zeros(categories))
#             cats[-1][item] =1.0
#         cat_sequences.append(cats)
#     return np.array(cat_sequences)

In [18]:
ntrain_x[0], len(ntrain_x[0]), train_x[0], train_y[0], ntrain_y[0]

(array([4303, 8973, 1799,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [40]:
otrain_y = to_categorical(ntrain_y, len(tag2idx))
otrain_y[0].shape

(271, 47)

In [21]:
ntrain_y[0]

array([27, 27, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [32]:
3131*271

848501

In [27]:
otrain_y.shape

(848501, 271, 47)

In [45]:
model.fit(ntrain_x, to_categorical(ntrain_y, len(tag2idx)), batch_size=128, epochs= 5, validation_split=0.2)

Train on 2504 samples, validate on 627 samples
Epoch 1/5
2504/2504 [==============================] - 55s 22ms/step - loss: 0.2939 - acc: 0.9170 - val_loss: 0.2896 - val_acc: 0.9179
Epoch 2/5
2504/2504 [==============================] - 57s 23ms/step - loss: 0.2849 - acc: 0.9180 - val_loss: 0.2827 - val_acc: 0.9205
Epoch 3/5
2504/2504 [==============================] - 56s 22ms/step - loss: 0.2791 - acc: 0.9203 - val_loss: 0.2785 - val_acc: 0.9226
Epoch 4/5
2504/2504 [==============================] - 57s 23ms/step - loss: 0.2740 - acc: 0.9222 - val_loss: 0.2736 - val_acc: 0.9227
Epoch 5/5
2504/2504 [==============================] - 57s 23ms/step - loss: 0.2716 - acc: 0.9238 - val_loss: 0.2673 - val_acc: 0.9281


In [22]:
len(ntrain_y)

3131

In [23]:
len(tag2idx)

47

In [24]:
848501/3131

271

In [25]:
(ntrain_x).shape, (ntrain_y).shape


((3131, 271), (3131, 271))

In [38]:
sequences =ntrain_y
categories =len(tag2idx)

def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        #print(s)
        cats =[]
        for item in s:
            #print(item)
            #print(np.zeros(categories))
            cats.append(np.zeros(categories))
            cats[-1][item] =1.0
    #         print(np.array(cats).shape)
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [27]:
np.array(cat_sequences[0]).shape

(271, 47)

In [37]:
np.array(cat_sequences).shape

(3131, 271, 47)

## Evaluation

In [44]:
ntest_y[0], ntest_x[0]

(array([30, 13, 26,  8, 27, 27,  6, 17, 17, 25, 22, 12, 26, 38, 26, 13, 17,
        30, 12, 38, 38, 27,  6, 17, 26, 20,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 

In [48]:
scores = model.evaluate(ntest_x, to_categorical(ntest_y, len(tag2idx)))
print("{}: {}".format(model.metrics_names[1],(scores[1] * 100)))


783/783 [==============================] - 5s 6ms/step
acc: 92.8164449916


In [50]:
predictions = model.predict(ntest_x)
#print(predictions, predictions.shape)

In [51]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [53]:
preds = logits_to_tokens(predictions, {i: t for t, i in tag2idx.items()})

In [56]:
len(preds), len(ntest_x)

(783, 783)

In [63]:
for q,p in zip(test_x,preds):
    #print("input: {}\n".format(q))
    print("output: {}\n".format(a))
    print("predcitions: {}\n =".format(p))

output: [u'DT' u'JJ' u'NN' u'VBZ' u'VBG' u'RP' u'IN' u'NNP' u'NNP' u'IN' u'PRP'
 u'VBZ' u'-NONE-' u'IN' u'NNP' u'NNP' u',' u'CC' u'JJ' u'NNS' u'VB'
 u'-NONE-' u'TO' u'VB' u'DT' u'NNP' u'DT' u'NN' u'-NONE-' u'TO' u'VB'
 u'RP' u'DT' u'NNS' u'WRB' u'VBG' u'VBZ' u'RB' u'JJ' u'-NONE-' u'.']

predcitions: [u'NNP', u'NNP', u'NNP', u'NNP', u'NNP', u'NNP', u'NN', u'NN', u'NN', u'NN', u'NN', u'NN', u'NN', u'IN', u'NN', u'IN', u'NN', u'NN', u'NN', u'NN', u'NN', u'NN', u'NN', u'NN', u'NN', u'.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>'


predcitions: [u'NNP', u'NNP', u'NNP', u'NNP', u'NNP', u'NN', u'NN', u'NN', u'DT', u'NN', u'NN', u'IN', u'NN', u'NN', u'NN', u'IN', u'NN', u'IN', u'NN', u'IN', u'NN', u'IN', u'NN', u'NN', u'IN', u'IN', u'NN', u'IN', u'NN', u'NN', u'NN', u'NN', u'NN', u'NN', u'.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>'